### mlexperience - similar news stores

In [26]:
import pandas as pd  #For python data frame
import pymongo #For mongoDB
#important
#run guardian.py first to create the data set
#guardian.py 
#start_date = date.today() - timedelta(2)  -  2 days from the current day
#end_date = date.today() - timedelta(1) - end 1 day from the current day
#to get a longer period change the timedelta(x) 


#Create connection to the MongoDB. Note the MongoDB must be running.
conn=pymongo.MongoClient('localhost',27017)
db = conn.mlexperience  #This database needs to be changed according to date of scrape
#Robomonogo 0.9.0 is useful to see the database and json structures of documents
#https://robomongo.org/

print(db.news.count())  #Print out the number of records in the database, visual check on connection.

df = pd.DataFrame(list(db.news.find())) #Copy the reports to the dataframe and close the connection
conn.close() 
    

247


In [27]:
df[:1]

,_id,content_type,date,link,ml_id,publication,published_by,simlist,summary,tags,text,title,topic
0,59386401464e070ec643b2f8,article,2017-05-08T23:53:19Z,https://gu.com/p/6dj59,world/2017/may/09/philippines-senator-tells-un...,theguardian.com,AUS,,Ally of Rodrigo Duterte employs Trump-speak to...,[{'Url': 'https://www.theguardian.com/world/ph...,There has been no new wave of killings prompte...,Philippines senator tells UN reports of drug w...,/World news


In [28]:
data1 = pd.DataFrame(data=df)
data1 = data1[['ml_id','topic','text']]
import numpy as np
#normalize machine learning related matrix
#data1.apply(lambda x: (x - np.mean(x) / (np.max(x) - np.min(x))))
#normalize
y = data1[['topic']]
x = data1[['text']]


In [29]:
def sim_doc(documents):
    from gensim import corpora, models, similarities
    documents = list(df['text'].values)
    # remove common words and tokenize
    stoplist = set('for a of the and to in'.split())
    texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in documents]

    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in texts:
         for token in text:
            frequency[token] += 1
    texts = [[token for token in text if frequency[token] > 0]
          for text in texts]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    tfidf = models.TfidfModel(corpus)
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=len(dictionary))
    return index, dictionary, tfidf

In [37]:
def find_stats(indexed, doc, dictionary, tfidf):
    vec_bow = dictionary.doc2bow(doc.lower().split())   
    sims = indexed[tfidf[vec_bow]]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    docs = list()
    for t in range(0,len(sims)):
        doc, score = sims[t]
        if (score > .10 and score!= 1.0):
            docs.append(doc)
    
    if (len(docs) > 0):
            return (docs)
    else:
            return 0,0,0
    return   

In [38]:
indexer, dictionary, tfidf = sim_doc(list(df['text'].values))

In [41]:
finder = 90
try:
    doc_find = df['text'][finder]
    stats = find_stats(indexer, doc_find, dictionary, tfidf)
    #print('Story: ', df['text'][finder])
    print(stats)
except KeyError:
    print('Not in memory')

[90, 226, 201]


In [44]:
doc_find = df['ml_id'][90]
doc_find1 = df['ml_id'][201]
doc_find2 = df['ml_id'][226]

In [45]:
print(doc_find, doc_find1, doc_find2)

lifeandstyle/2017/may/08/first-fire-woman-josephine-reynolds-1982 education/2017/may/08/nus-president-malia-bouattia-it-was-the-most-difficult-year-of-my-life tv-and-radio/2017/may/08/caitlyn-jenner-bruce-transitioning-kardashians-reality-tv-star
